In [ ]:
#|hide
# This notebook documents tsai_rs - Time Series Deep Learning in Rust

<div align="center">
<h1>tsai-rs</h1>
<p><strong>Time Series Deep Learning in Rust</strong></p>
</div>

# tsai-rs

A high-performance time series deep learning library written in Rust with Python bindings.

## Description

> State-of-the-art Deep Learning library for Time Series and Sequences - implemented in Rust for maximum performance.

`tsai_rs` is a Rust implementation of time series deep learning models, inspired by the popular Python `tsai` library. It provides:

- High-performance Rust implementations using the `burn` deep learning framework
- Python bindings via PyO3 for seamless integration with Python workflows
- Support for Apple Silicon (MLX backend) and CUDA GPUs
- State-of-the-art models: InceptionTimePlus, ResNetPlus, PatchTST, TSTPlus, RNNPlus, MiniRocket, and more

## Installation

### From PyPI

```bash
pip install tsai_rs
```

### From Source

```bash
git clone https://github.com/timeseriesAI/tsai-rs
cd tsai-rs
pip install maturin
maturin develop --release
```

## Quick Start

### Basic Usage with Python Bindings

In [ ]:
# Import tsai_rs Python bindings
import tsai_rs
import numpy as np

### Loading UCR Datasets

In [ ]:
# Load a UCR classification dataset
X_train, y_train, X_test, y_test = tsai_rs.get_UCR_data('ECG200')

print(f"Training samples: {X_train.shape}")
print(f"Test samples: {X_test.shape}")
print(f"Number of classes: {len(np.unique(y_train))}")

### Data Preprocessing

In [ ]:
# Standardize time series data
X_train_std = tsai_rs.ts_standardize(X_train)
X_test_std = tsai_rs.ts_standardize(X_test)

print(f"Mean after standardization: {np.mean(X_train_std):.6f}")
print(f"Std after standardization: {np.std(X_train_std):.6f}")

### Data Augmentation

In [ ]:
# Add Gaussian noise for data augmentation
X_noisy = tsai_rs.add_gaussian_noise(X_train_std, scale=0.1)

# Magnitude scaling
X_scaled = tsai_rs.mag_scale(X_train_std, scale=0.2)

### Time Series Imaging

In [ ]:
# Compute Gramian Angular Summation Field
gasf = tsai_rs.compute_gasf(X_train_std)
print(f"GASF shape: {gasf.shape}")

# Compute Gramian Angular Difference Field
gadf = tsai_rs.compute_gadf(X_train_std)
print(f"GADF shape: {gadf.shape}")

# Compute Recurrence Plot
rp = tsai_rs.compute_recurrence_plot(X_train_std)
print(f"Recurrence Plot shape: {rp.shape}")

## Available Models

tsai-rs provides high-performance implementations of state-of-the-art time series models:

### Convolutional Models
- **InceptionTimePlus** - Enhanced InceptionTime with improved architecture
- **ResNetPlus** - Enhanced ResNet for time series

### Transformer Models
- **TSTPlus** - Time Series Transformer with enhancements
- **PatchTST** - Patch-based Time Series Transformer (ICLR 2023)

### Recurrent Models
- **RNNPlus** - LSTM/GRU with modern improvements

### Kernel-based Models
- **MiniRocket** - Fast random convolutional kernel transform

### Model Configurations

In [ ]:
# InceptionTimePlus configuration
inception_config = tsai_rs.InceptionTimePlusConfig(
    c_in=1,           # Number of input channels
    c_out=2,          # Number of output classes
    seq_len=96,       # Sequence length
    nf=32,            # Number of filters
    depth=6,          # Network depth
    bottleneck=True,  # Use bottleneck layers
    residual=True     # Use residual connections
)

# ResNetPlus configuration
resnet_config = tsai_rs.ResNetPlusConfig(
    c_in=1,
    c_out=2,
    seq_len=96,
    nf=64
)

# PatchTST configuration
patchtst_config = tsai_rs.PatchTSTConfig(
    c_in=1,
    c_out=2,
    seq_len=96,
    patch_len=16,
    stride=8,
    n_heads=4,
    d_model=128,
    d_ff=256,
    n_layers=3,
    dropout=0.1
)

# TSTPlus configuration  
tst_config = tsai_rs.TSTConfig(
    c_in=1,
    c_out=2,
    seq_len=96,
    n_heads=4,
    d_model=64,
    d_ff=128,
    n_layers=2,
    dropout=0.1
)

# RNNPlus configuration
rnn_config = tsai_rs.RNNPlusConfig(
    c_in=1,
    c_out=2,
    seq_len=96,
    hidden_size=128,
    n_layers=2,
    rnn_type='LSTM',  # or 'GRU'
    bidirectional=True,
    dropout=0.1
)

# MiniRocket configuration
minirocket_config = tsai_rs.MiniRocketConfig(
    c_in=1,
    seq_len=96,
    num_features=10000
)

## Training Configuration

In [ ]:
# Learner configuration for training
learner_config = tsai_rs.LearnerConfig(
    lr=1e-3,
    epochs=100,
    batch_size=64,
    optimizer='AdamW',
    weight_decay=1e-2
)

# One Cycle Learning Rate Scheduler
scheduler = tsai_rs.OneCycleLR(
    max_lr=1e-3,
    epochs=100,
    pct_start=0.3,
    div_factor=25.0,
    final_div_factor=10000.0
)

## Example: Binary Classification with InceptionTimePlus

In [ ]:
# Load dataset
X_train, y_train, X_test, y_test = tsai_rs.get_UCR_data('ECG200')

# Standardize
X_train = tsai_rs.ts_standardize(X_train)
X_test = tsai_rs.ts_standardize(X_test)

# Configure model
config = tsai_rs.InceptionTimePlusConfig(
    c_in=X_train.shape[1],
    c_out=len(np.unique(y_train)),
    seq_len=X_train.shape[2]
)

# Configure training
learner = tsai_rs.LearnerConfig(
    lr=3e-4,
    epochs=100,
    batch_size=64
)

print(f"Model configured for {config.c_out} classes with sequence length {config.seq_len}")

## Example: Multivariate Classification

In [ ]:
# Load multivariate dataset
X_train, y_train, X_test, y_test = tsai_rs.get_UCR_data('LSST')

# Standardize by sample
X_train = tsai_rs.ts_standardize(X_train, by_sample=True)
X_test = tsai_rs.ts_standardize(X_test, by_sample=True)

# Configure TSTPlus for multivariate data
config = tsai_rs.TSTConfig(
    c_in=X_train.shape[1],
    c_out=len(np.unique(y_train)),
    seq_len=X_train.shape[2],
    n_heads=8,
    d_model=128
)

print(f"Multivariate model: {config.c_in} variables, {config.c_out} classes")

## Analysis Tools

In [ ]:
# Compute confusion matrix
y_true = np.array([0, 1, 0, 1, 0, 1])
y_pred = np.array([0, 1, 0, 0, 0, 1])
cm = tsai_rs.confusion_matrix(y_true, y_pred)
print(f"Confusion Matrix:\n{cm}")

# Find top losses for analysis
losses = np.array([0.1, 0.8, 0.2, 0.9, 0.15, 0.7])
top_idx = tsai_rs.top_losses(losses, k=3)
print(f"Top 3 loss indices: {top_idx}")

## Performance Benefits

tsai-rs provides significant performance improvements over pure Python implementations:

- **Rust Core**: Zero-cost abstractions and memory safety
- **Burn Framework**: Efficient tensor operations with multiple backends
- **MLX Backend**: Native Apple Silicon support for M1/M2/M3 chips
- **CUDA Backend**: GPU acceleration for NVIDIA hardware
- **Parallel Processing**: Efficient multi-threaded data loading and augmentation

## Input Data Format

The input format for all time series models in tsai-rs is:

**[# samples x # variables x sequence length]**

This is consistent with the original tsai library and most deep learning frameworks for time series.

## Documentation

For more details, see the individual module notebooks:

- `002_utils.ipynb` - Utility functions
- `005_data.external.ipynb` - Loading external datasets
- `009_data.preprocessing.ipynb` - Data preprocessing
- `010_data.transforms.ipynb` - Data transformations
- `012_data.image.ipynb` - Time series imaging (GASF, GADF, Recurrence Plots)
- `036_models.InceptionTimePlus.ipynb` - InceptionTimePlus model
- `034_models.ResNetPlus.ipynb` - ResNetPlus model
- `050_models.TSTPlus.ipynb` - TSTPlus model
- `050b_models.PatchTST.ipynb` - PatchTST model
- `043_models.RNNPlus.ipynb` - RNNPlus model
- `055_models.MINIROCKET.ipynb` - MiniRocket model

## Citing tsai-rs

If you use tsai-rs in your research, please cite:

```bibtex
@software{tsai_rs,
    title = {tsai-rs: Time Series Deep Learning in Rust},
    author = {timeseriesAI},
    year = {2024},
    url = {https://github.com/timeseriesAI/tsai-rs}
}
```